# SEV-CV Mathematics and Training Overview

This notebook explains the model, objectives, and the evolutionary controller.

## Objectives (GAN hinge)
- Discriminator: L_D = E[max(0, 1 - D(x))] + E[max(0, 1 + D(G(z)))]
- Generator: L_G = - E[D(G(z))]

Add-ons (future): R1 penalty, path length regularizer, wavelet loss, CLIP alignment for conditional.

## Transformer Block
- LayerNorm + Multi-Head Attention + residual
- LayerNorm + MLP (GELU) + residual

Tokens are H×W patches reshaped to N×C.

## Evolutionary Controller
- Individuals contain policy (lr, ema), micro-arch (heads, depth), and z seed.
- Mutate: jitter lr, ema; heads/depth ±1.
- Select: rank by fitness proxy (diversity/variance placeholder).

## Training Loop Sketch
for step in range(K):
  - sample real batch
  - update D on real/fake (hinge)
  - update G to fool D
Every T steps:
  - evolve policies and micro-arch (phase 1+)

In [ ]:
# Hinge loss demo
import tensorflow as tf
batch = 8
logits_real = tf.random.normal([batch, 1])
logits_fake = tf.random.normal([batch, 1])
d_loss = tf.reduce_mean(tf.nn.relu(1. - logits_real)) + tf.reduce_mean(tf.nn.relu(1. + logits_fake))
g_loss = -tf.reduce_mean(logits_fake)
d_loss, g_loss

In [ ]:
# Tiny attention example
import tensorflow as tf
B,N,C,H = 2, 16, 64, 4
x = tf.random.normal([B,N,C])
attn = tf.keras.layers.MultiHeadAttention(num_heads=H, key_dim=C//H)
y = attn(x, x)
y.shape